In [1]:
import numpy as np
from scipy.integrate import quad
import pygame 

from formula_functions import *
from animation_functions import *

from show_parameters_functions import *
from key_controls_functions import *
from formula_functions import *

from Data_classes.data_processing import DataProcessing

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
dd = DataProcessing()
dd.constants.screen_width

1500

In [3]:
def draw_explanations(data_processing,font_small, t_mth_list):
    
    explanations_layer = data_processing.mode_statuses_dict['Explanations layer'][0]
    explanations_layer.fill((0, 0, 0, 0))
    
    w = data_processing.get_curr_param('w')
    k = data_processing.get_curr_param('k')
    length = data_processing.get_curr_param('l')


#     half_screen_width = data_processing.constants.half_screen_width
#     half_screen_height = data_processing.constants.half_screen_height
    
    dots_params = data_processing.draw_dots_params
    const_center_point = dots_params[6]
    var_center_point = dots_params[7]
    
    center_point_width = const_center_point[0] + var_center_point[0]
    center_point_height = const_center_point[1] + var_center_point[1]
    
    PMD_x = get_nth_deg_x_derivative(0, 3.6796737050303396, 0.25, 0.29, 0.8)
    PMD_y = get_nth_deg_y_derivative(0, 3.6796737050303396, 0.25, 0.29, 0.8)
    
    PMD_x = x_transform(PMD_x, center_point_width, length)
    PMD_y = y_transform(PMD_y, center_point_height, length)
    
    PR_x = get_nth_deg_x_derivative(0, 5.856, 0.25, 0.29, 0.8)
    PR_y = get_nth_deg_y_derivative(0, 5.856, 0.25, 0.29, 0.8) 
    
    PR_x = x_transform(PR_x, center_point_width, length)
    PR_y = y_transform(PR_y, center_point_height, length)
    
    t_1_y = get_nth_deg_y_derivative(0, 1.083307811582687, 0.25, 0.29, 0.8)
    t_1_y = y_transform(t_1_y, center_point_height, length)
    
    pygame.draw.circle(explanations_layer, color='black', center=(PMD_x, PMD_y), radius=5)
    pygame.draw.circle(explanations_layer, color='black', center=(PR_x, PR_y), radius=4)
    
    pygame.draw.line(explanations_layer,start_pos = (0, t_1_y), end_pos = (data_processing.constants.screen_width,
                                                                           t_1_y),width = 1, color='black')
    pygame.draw.line(explanations_layer,start_pos = (PR_x, PR_y), end_pos = (PR_x,
                                                                           center_point_height),width = 1, color='black')
    pygame.draw.line(explanations_layer,start_pos = (PMD_x, PMD_y), end_pos = (PMD_x,
                                                                           center_point_height),width = 1, color='black')


    main_font = pygame.font.SysFont("Arial", 30)
    subscript_font = pygame.font.SysFont("Arial", 20)
    # bigg_letter_width = 50
    # bigg_letter_height = 10
    
#     index_letter_width = bigg_letter_width +12
#     index_letter_height = bigg_letter_height + 13
    
    
    pmd = main_font.render("PMD", True, (0, 0, 0))
    pr = main_font.render("PR", True, (0, 0, 0))
    t = main_font.render("t", True, (0, 0, 0))
    one = subscript_font.render("1", True, (0, 0, 0))
    pr_sub = subscript_font.render("pr", True, (0, 0, 0))
    pmd_sub = subscript_font.render("pmd", True, (0, 0, 0))
    X = main_font.render("x", True, (0, 0, 0))
#     a = main_font.render("a", True, (0, 0, 0))
    
# #     index_letter = subscript_font.render("l", True, (0, 0, 0))
    
# #     text = font_small.render('\u03B8', True, (0, 0, 0))

#     delta_phi = main_font.render("\u0394\u03C6", True, (0, 0, 0)) 
    explanations_layer.blit(pmd, (PMD_x - 20, PMD_y - 50))
    explanations_layer.blit(pr, (PR_x - 40, t_1_y-40))

    explanations_layer.blit(t, (center_point_width + 20, t_1_y-40))
    explanations_layer.blit(one, (center_point_width + 30, t_1_y-25))
    
    explanations_layer.blit(X, (PR_x-5, center_point_height + 15))
    explanations_layer.blit(pr_sub, (PR_x+7, center_point_height + 27))
    
    explanations_layer.blit(X, (PMD_x-5, center_point_height + 15))
    explanations_layer.blit(pmd_sub, (PMD_x+7, center_point_height + 27))
    
#     explanations_layer.blit(delta_phi, (center_point_width+80, center_point_height-230))
   
   

In [ ]:
def main_loop():

    '''
    Initializations
    
    '''
    
    pygame.init()

    line_color = (185, 185, 185) # Color of the coordinate line
    
    clock = pygame.time.Clock()
    
    
    data_processing = DataProcessing()
    
    data_processing.constants.text_spacing = 45
    data_processing.constants.text_unit = data_processing.constants.text_spacing / 1.3
    
    font_small = pygame.font.Font(None, data_processing.constants.text_spacing)

    screen_width, screen_height, FPS = data_processing.constants.screen_width, \
                                       data_processing.constants.screen_height, \
                                        data_processing.constants.FPS
    
    # Layers
    win = pygame.display.set_mode((screen_width, screen_height)) # Main animation layer

    # data_processing.reset_dicts()
    

    data_processing.constants.constants_dict['t'] = 6.62
    data_processing.constants.constants_dict['x'] = -0.86
    data_processing.constants.constants_dict['v'] = 0.25
    data_processing.constants.constants_dict['w'] = 0.29
    # data_processing.constants.constants_dict['b'] = -3.9
    data_processing.constants.constants_dict['k'] = 0.8
    # data_processing.constants.constants_dict['a'] = 59
    data_processing.variables.variables_dict['l'] = 543
    data_processing.variables.variables_dict['c'] = [500, 180]
    
  
    # data_processing.constants.constants_dict['t'] = 2.5908
    # data_processing.constants.constants_dict['x'] = -2.49
    # data_processing.constants.constants_dict['v'] = 3
    # data_processing.constants.constants_dict['w'] = 2
    # data_processing.constants.constants_dict['b'] = -3.9
    # data_processing.constants.constants_dict['k'] = 3
    # data_processing.constants.constants_dict['a'] = 59
    # data_processing.variables.variables_dict['l'] = 51
    # data_processing.variables.variables_dict['c'] = [0, 240]
    

    t_mth_aproxim_list = [] # caches the already computed approximation values

    create_background(*data_processing.background_params, font_small)

    draw_line(*data_processing.line_params)
    
    
    draw_derivatives(*data_processing.derivatives_params)

    draw_spiral(*data_processing.spiral_params)
    
    t_nth_list = calc_y_intersects_t(*data_processing.y_intersects_t_params)

    # Initial drawing of the y-intersection points
    draw_dots(*data_processing.draw_dots_params, t_nth_list, 'Y-intersects')

    show_parameters(data_processing, font_small)
    
    show_mode_statuses(data_processing, font_small)
    
    
    data_processing.blit_layers(win)
    
    pygame.display.update()
 
    running = True
    
    while running:
        
        clock.tick(data_processing.constants.FPS)

        # All updates booleans must be set to False
        data_processing.booleans.reset_to_false()
      
        # Check and handle for key control events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            else:
    
                handle_ctrl_commands(event, data_processing)
                
                handle_steps_variables(event, data_processing)
  
                handle_switch_commands(event, data_processing)
                
    
                if data_processing.mode_statuses_dict['Algorithm mode'][1]:
                    t_mth_aproxim_list = handle_algorithm_mode_controls(event, data_processing, t_mth_aproxim_list)
                    
                else:

                    handle_shift_key_commands(event, data_processing)

                if data_processing.booleans.update_booleans_dict['is_turn_off'] is not None:
                    
                    data_processing.mode_statuses_dict['Algorithm mode'][0].fill((0, 0, 0,0))
                    data_processing.mode_statuses_dict['Algorithm data mode'][0].fill((0, 0, 0, 0))
                    t_mth_aproxim_list.clear()
                    data_processing.algorithm_vars.reset_dict()
                    data_processing.mode_statuses_dict['Derivatives'][1] = False

        handle_key_commands(data_processing)

        if data_processing.booleans.update_booleans_dict['update_screen']:

            if data_processing.mode_statuses_dict['Algorithm mode'][1]:
                
                data_processing.mode_statuses_dict['Derivatives'][1] = False
                
            draw_derivatives(*data_processing.derivatives_params)
            
            if data_processing.booleans.update_booleans_dict['shift_coords'] \
                or data_processing.booleans.update_booleans_dict['reset_background']:
               
                # Update coordinate system
                create_background(*data_processing.background_params, font_small)

            
            if data_processing.booleans.update_booleans_dict['update_spiral'] \
                or data_processing.booleans.update_booleans_dict['is_t_diagram_change']:
                draw_spiral(*data_processing.spiral_params)

            if data_processing.booleans.update_booleans_dict['update_line']:         
                
                draw_line(*data_processing.line_params)

            # A list contains the exact t-parameters for the intersection points of the spiral with y-axis
            t_nth_list = calc_y_intersects_t(*data_processing.y_intersects_t_params)
           
            
            # Draw current y intersection points if y-intersects mode is `on`
            if data_processing.mode_statuses_dict['Y-intersects'][1]:
             
                draw_dots(*data_processing.draw_dots_params, t_nth_list, 'Y-intersects')

            # A list contains the exact t-parameters for the intersection points of the spiral with the vertical line
            t_mth_list = calc_line_intersections_t(*data_processing.line_intersections_t_params, t_nth_list)
            
            # Draw current line intersection points if line-intersects mode is `on`
            if data_processing.mode_statuses_dict['Line-intersects'][1]:
                draw_dots(*data_processing.draw_dots_params, t_mth_list, 'Line-intersects')
      
            # draw_circle_and_vector(data_processing, font_small)
                
            # Show steps variables if the steps change mode is `on`    
            show_steps_variables(*data_processing.show_steps_params, font_small)
            
            # Show the current statuses of the spiral and line parameters
            show_parameters(data_processing, font_small)  
            
            # Show the current statuses of the mode parameters
            show_mode_statuses(data_processing, font_small)
            
            # show_circle_mode_parameters(data_processing, font_small)

            # Show the algorithm steps towards the interesection point
            if data_processing.mode_statuses_dict['Algorithm mode'][1]:

                t_mth_aproxim_list, data_processing.algorithm_vars.algorithm_vars_dict['total_n'] = draw_algorithm_steps(*data_processing.draw_algorithm_params, 
                                                          t_nth_list, t_mth_aproxim_list)
            
                x, y = 'None', 'None'
                if t_nth_list:
                    nth_t = t_nth_list[data_processing.algorithm_vars.algorithm_vars_dict['n']]
                else:
                    nth_t = 0
                mth_t = 0
                if len(t_mth_aproxim_list) > 1:
            
            
                    mth_t = t_mth_aproxim_list[data_processing.algorithm_vars.algorithm_vars_dict['m']]
            
                    x, y = calc_single_t_aproxim(*data_processing.calc_sigle_t_approxim_params, mth_t, transform=False)
                
                # Show the current statuses of the m_th aproximation point
                show_algorithm_rows_and_cols(data_processing, nth_t, mth_t,  x, y, font_small)
                
            draw_explanations(data_processing, font_small, t_mth_aproxim_list)
            
            # print('l: ', data_processing.variables.variables_dict['l'])
#             print('l curr param: ', data_processing.get_curr_param('l'))
        
#             print('c: ', data_processing.variables.variables_dict['c'])
            
            
            # Attache the new drawings on the screen    
            data_processing.blit_layers(win)
            
            pygame.display.update()

        
    # Exit
    pygame.quit()
    
main_loop()

Space
